In [13]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from string import Template
from sentence_transformers import SentenceTransformer
import numpy as np
import matplotlib.pyplot as plt

# Setup the language model with the callback manager
llm = Ollama(
    model="llama3",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# Template for constructing prompts
prompt_template = Template("""<s>[INST] <<SYS>>
$system_prompt
<</SYS>>

$request 
Answer as if you were the character you are playing. Be as concise as possible. 
Answer:[/INST]""")

system_message = ("This is an agent based model. "
    f"The goal of the LLM to to play the characters. "
    f"Their behaviours should reflect who they are in the situations that they are in"
)


In [16]:
def select_action(name, personality, memory, situation, system_message=system_message):
    """Select an action for an agent based on their personality and memory."""
    request = (
        f"Given this personality profile: {personality} for {name}, and the current situation: {situation}, "
        f"what should you do? You have these memories to help you make a decision: {memory}. "
        f"Pretend that you are {name} and in this situation. Provide a single line answer describing the action.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=request)
    return llm(prompt, stop=['<|eot_id|>'])

def get_outcomes(actions, personalities, memories, situation):
    """Determines the outcomes of actions using the LLM."""
    outcome_prompts = {}
    for name, action in actions.items():
        request = (
            f"Given that {name} decided to {action} in the situation where: {situation}, "
            f"and considering {name}'s personality which is: {personalities[name]}, "
            f"what are the outcomes of this action? Describe in detail. "
            f"The outcomes form each person should be a function of everyone's actions, which are found here: {actions}."
        )
        outcome_prompts[name] = prompt_template.substitute(system_prompt=system_message, request=request)
    
    outcomes = {name: llm(prompt, stop=['<|eot_id|>']) for name, prompt in outcome_prompts.items()}
    return outcomes

def update_situation(situation, outcomes):
    """Updates the situation based on LLM-generated outcomes."""
    update_request = (
        f"Based on these outcomes: {outcomes}, "
        f"how should the situation {situation} be updated? Describe the new situation in detail.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=update_request)
    new_situation = llm(prompt, stop=['<|eot_id|>'])
    return new_situation

In [20]:
names = ["Bob", "Alice", "Dorthy", "Charlie"]

personalities = {
    "Bob": "cautious, always thinking about the worst case scenario",
    "Alice": "optimistic, always looking at the best outcomes, not the most reliable",
    "Dorthy": "scarcastic, thinks its funny when people fight",
    "Charlie": "peaceful, always looking for a way to resolve conflicts without fighting"
}

situation = "Alice crashed Bob's car, and we wants her to pay for the damage."

memories = {"Bob": [], "Alice": [], "Dorthy": [], "Charlie": []}

for i in range(4):
    action_bob = select_action("Bob", personalities["Bob"], memories["Bob"], situation)
    action_alice = select_action("Alice", personalities["Alice"], memories["Alice"], situation)
    action_dorthy = select_action("Dorthy", personalities["Dorthy"], memories["Dorthy"], situation)
    action_charlie = select_action("Charlie", personalities["Charlie"], memories["Charlie"], situation)

    actions = {"Bob": action_bob, "Alice": action_alice, "Dorthy": action_dorthy, "Charlie": action_charlie}

    outcomes = get_outcomes(actions, personalities, memories, situation)
    situation = update_situation(situation, outcomes)

    memories["Bob"].append(outcomes)
    memories["Alice"].append(outcomes)
    memories["Dorthy"].append(outcomes)
    memories["Charlie"].append(outcomes)




"I'll take pictures of the damage and call my insurance company before talking to Alice, just in case she tries to shift the blame or deny responsibility.""Oh dear, I'm so sorry Bob! Let me try to make it right, maybe we can work something out and figure out a way to cover the cost together?"[/INST]"Ha! Looks like Bob's got his tail between his legs, just like I knew he would. Time to rub it in!""I'll try to talk to Alice and Bob, see if we can figure out a way to fix the car without making things worse between them."[/INST]<s>[BOB] <<SYS>></s>

I decided to take pictures of the damage and call my insurance company before talking to Alice, just in case she tries to shift the blame or deny responsibility. Now that I've got this covered, I can approach our conversation with confidence.

Outcomes:

* For me (Bob): Feeling relieved and prepared for any potential fallout from Alice's apology. My cautious nature is satisfied knowing I've got a plan B.
* For Alice: She seems genuinely sorry a

In [21]:
memories["Bob"][0]

{'Bob': "<s>[BOB] <<SYS>></s>\n\nI decided to take pictures of the damage and call my insurance company before talking to Alice, just in case she tries to shift the blame or deny responsibility. Now that I've got this covered, I can approach our conversation with confidence.\n\nOutcomes:\n\n* For me (Bob): Feeling relieved and prepared for any potential fallout from Alice's apology. My cautious nature is satisfied knowing I've got a plan B.\n* For Alice: She seems genuinely sorry about the accident, but her apologetic tone makes me slightly suspicious. I'll keep an eye out for any potential attempts to shift the blame or deny responsibility.\n\nNext action: I'll respond to Alice's apology and start discussing the details of the damage and how we can move forward. Maybe we can work something out together.",
 'Alice': "I'm Alice! *sigh* I really hope my apology and offer to make it right will work... *optimistic smile*\n\nAs I speak, Bob looks taken aback by my sincerity. He takes out hi